## Building a Question-Answering Chatbot with Gradio

In this exercise, we will use LangChain and OpenAI's ChatGPT API to build a chatbot that can answer questions using information from a set of source documents.

LangChain is a framework for building apps using language models. There are many off-the-shelf chains - sequences of calls to components, including other chains - that make it easy to get something going really quickly.

#### Part 1: Chatbot Interface
First let's build the interface using Gradio and get the skeleton of the chatbot in place. 

For the time being, we will use `send_chat_message()` to simulate the chatbot's responses.

In [1]:
import gradio as gr
import random

def send_chat_message(prompt):
    return random.choice(["How are you?", "Have a great day!", "I'm very hungry"])

def respond(message, chat_history):
    bot_message = send_chat_message(message)
    chat_history.append((message, bot_message))
    return "", chat_history

with gr.Blocks(title = "Datajam DocBot") as demo:
    with gr.Row():
        gr.Markdown(
        """
        <h1 align='left'> <img src='file/assets/datajam-logo.png' height='50' width='129'> </h1>
        <h1 align="center"> DocBot 🤖💬 </h1>
        <p align='center' style='font-size: 18px;'> Meet ✨ DocBot ✨, an intelligent chatbot that helps answer your deepest questions about the Unbounce universe. </p>
        """)
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Have a question? How can I help? Start the chat.", show_label=False)
    clear = gr.ClearButton([msg, chatbot])
    msg.submit(respond, [msg, chatbot], [msg, chatbot])
demo.launch()

/Users/sampoornabiswas/.pyenv/versions/3.10.5/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7870

To create a public link, set `share=True` in `launch()`.


#### Part 2: The AI Chatbot
Now we can build the question-answering (QA) chatbot component using large language models (LLM). We won't be training our own model, instead we shall use OpenAI's [chat API](https://platform.openai.com/docs/guides/gpt) to query their pre-trained model.

##### Using Environment Files
In the last demo, we put our secret key in plaintext in the notebook itself. This is not very secure. If we wanted to share this notebook with someone else, we would always have to remember to remove the secret key.

An alternative is to use environment files and simply use the notebook to read the values from that file. To do this,
1. Create a new blank file in the same directory and call it `.env`
2. Write `OPENAI_API_KEY=<your secret key>`
3. Now we can read these environment variables using `load_dotenv()`

In [ ]:
import openai
import os
from dotenv import load_dotenv

# Load API keys
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')

##### Chatbot Pipeline
The pipeline for converting raw unstructured data into a QA chain looks like this:

1. **Loading**: First we need to load our data. Unstructured data can be loaded from many sources. The LangChain integration hub has the full set of loaders. Each loader returns data as a LangChain Document.
2. **Splitting**: Text splitters break documents into chunks of specified size
3. **Storage**: Storage (e.g., often a vectorstore) will house and often embed the chunks
4. **Retrieval**: The app retrieves splits from storage (e.g., often with similar embeddings to the input question)
5. **Generation**: An LLM produces an answer using a prompt that includes the question and the retrieved data
6. **Conversation (Extension)**: Hold a multi-turn conversation by adding Memory to your QA chain.

##### Step 1: Load
The [Unstructured PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf#using-unstructured) reads in PDFs and returns a LangChain Document object. 

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader

DATA_DIRECTORY = "../data"
path = os.path.join(DATA_DIRECTORY, "content_library.pdf")
loader = UnstructuredPDFLoader(path)
documents = loader.load()

##### Step 2-4: Split, Store, Retrieve
The rest of the pipeline can be wrapped into a single object: `VectorstoreIndexCreator` which splits, embeds, stores and retrieves data.

In [ ]:
from langchain.indexes import VectorstoreIndexCreator

index = VectorstoreIndexCreator(
    vectorstore_kwargs={"persist_directory": DATA_DIRECTORY}
).from_documents(documents)

##### Step 5-6: Generate and Converse
Chains are *the* main feature of LangChain, enabling us to build complex applications by linking up different components. The components can be language models, other chains, document retrievers, prompts, utils, etc. There are a number of pre-built chains to help you get started.

For this workshop, let's use a [Conversational Retrieval Chain](https://api.python.langchain.com/en/latest/chains/langchain.chains.conversational_retrieval.base.ConversationalRetrievalChain.html). It answers new questions using chat history (a list of previous messages) and source documents.

The three main components of this chain are:

1. Using the chat history and the new question to create a “standalone question”: This way, the question can be passed into the retrieval step to fetch relevant documents. For eg., consider a follow-up question like "Where can I find it?". If only the new question was passed in, then language model would have no idea what 'it' is. If the whole conversation was passed in, there may be unnecessary information that would distract from retrieval.

2. Retrieving relevant documents using this new question.

3. Obtaining the final response: The retrieved documents are passed to an LLM along with the new question and it uses them to generate an answer.

In [ ]:
from langchain.chains import ConversationalRetrievalChain
from langchain.chat_models import ChatOpenAI

chain = ConversationalRetrievalChain.from_llm(
    llm=ChatOpenAI(model="gpt-4", temperature=0.9),
    retriever=index.vectorstore.as_retriever(search_kwargs={"k": 1}),
    condense_question_llm = ChatOpenAI(temperature=0, model='gpt-3.5-turbo'),
)

Now let's put this all together into a new method `send_chat_message()` that calls the chain with the user question.

In [ ]:
def send_chat_message(prompt, chat_history, chain):
    """
    Converts Gradio's chat history format to LangChain's expected 
    format and queries the chat engine for a response.
    """
    langchain_history = [(msg[0], msg[1]) for msg in chat_history]

    result = chain({"question": prompt, "chat_history": langchain_history})
    return result["answer"]

def respond(message, chat_history):
    bot_message = send_chat_message(message, chat_history, chain)
    chat_history.append((message, bot_message))
    return "", chat_history

In [ ]:
with gr.Blocks(title = "Datajam DocBot") as demo:
    with gr.Row():
        gr.Markdown(
        """
        <h1 align='left'> <img src='file/assets/datajam-logo.png' height='50' width='129'> </h1>
        <h1 align="center"> DocBot 🤖💬 </h1>
        <p align='center' style='font-size: 18px;'> Meet ✨ DocBot ✨, an intelligent chatbot that helps answer your deepest questions about the Unbounce universe. </p>
        """)
    chatbot = gr.Chatbot()
    msg = gr.Textbox(placeholder="Have a question? How can I help? Start the chat.", show_label=False)
    clear = gr.ClearButton([msg, chatbot])

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    gr.Examples(
        examples=[
            "How do I add my domain to Unbounce?",
            "What is a compelling call to action?",
            "How do I add a user to my account?",
            "How does Smart Traffic compare to A/B testing?",
            "What are the options within Account Management?",
            "What is a landing page?"
        ],
        inputs=msg
    )
demo.launch()